In [1]:
import csv
import numpy as np
import pandas as pd

In [2]:
crime_csv_url = 'https://www.denvergov.org/media/gis/DataCatalog/crime/csv/crime.csv'
offense_codes_csv_url = 'https://www.denvergov.org/media/gis/DataCatalog/crime/csv/offense_codes.csv'

In [3]:
df = pd.read_csv(crime_csv_url)
offense = pd.read_csv(offense_codes_csv_url)

In [ ]:
df['FIRST_OCCURRENCE_DATE'] = pd.to_datetime(df['FIRST_OCCURRENCE_DATE'])
df['DATE'] = df['FIRST_OCCURRENCE_DATE'].dt.date
df['DATE'] = pd.to_datetime(df['DATE'])
df.info()

In [ ]:
df_offense_type = df.groupby(['DATE','NEIGHBORHOOD_ID','OFFENSE_CATEGORY_ID','OFFENSE_TYPE_ID']).agg({'INCIDENT_ID':'count'}).reset_index()
df_offense_type.head()


In [ ]:
df_crosstab = pd.crosstab([df_offense_type.NEIGHBORHOOD_ID, df_offense_type.OFFENSE_TYPE_ID], 
                          df_offense_type.DATE,  margins=True)
df_crosstab.head()

In [ ]:
df_neighborhood = df_offense_type.pivot_table(index='DATE', values='INCIDENT_ID',
                          columns='NEIGHBORHOOD_ID', aggfunc=np.sum,
                                             fill_value=0)
df_neighborhood.head()

In [ ]:

#_ = df_neighborhood.reset_index().plot(x='DATE', y=df_neighborhood.columns)

#plt.show()

In [ ]:
import time
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.grid_objs import Grid, Column

#py.init_notebook_mode(connected=True)

In [ ]:
trace_names = list(df_neighborhood.columns)
traces = []
for trace in trace_names:
    traces.append(go.Scatter(x=df_neighborhood.index, y=df_neighborhood[trace].values, name=trace, stackgroup='A'))

layout = dict(
    title='Time Series with Rangeslider',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = dict(data=traces, layout=layout)
py.iplot(fig, filename = "Time Series with Rangeslider")

In [ ]:
import datetime
today = datetime.datetime.today().date()

In [ ]:
#df_neighborhood.index = pd.to_datetime(df_neighborhood.index, unit='D')
df_neighborhood_month = df_neighborhood.resample('M').sum()
df_neighborhood_quarter = df_neighborhood.resample('Q').sum()
df_neighborhood_year = df_neighborhood.resample('Y').sum()

In [ ]:
_ = df_neighborhood_year.pct_change().drop(df_neighborhood_year.tail(1).index).plot(legend=False)

In [ ]:
df_neighborhood_rolling_month = df_neighborhood_month.drop(df_neighborhood_month.tail(1).index).rolling(3).mean()
#df_neighborhood_rolling30.sort_values(by='DATE', ascending=False).head()
_ = df_neighborhood_rolling_month.sort_values(by='DATE', ascending=False).head(36).plot.area(legend=False)

In [ ]:
neighborhoods = pd.DataFrame({'NEIGHBORHOOD_ID':df['NEIGHBORHOOD_ID'].unique()}) 

In [ ]:
df_neighborhood_month = df_neighborhood_month[list(df_neighborhood_month.max().sort_values(ascending=False).index)]
_ = df_neighborhood_month.plot.box(figsize=(12,5), rot=90)

In [ ]:
max_index = list(df_neighborhood_month.max().sort_values(ascending=False).index)

trace_names = list(max_index)
traces = []
for trace in trace_names:
    traces.append(go.Box(y=df_neighborhood[trace].values, name=trace))

data = traces
py.iplot(data)

In [ ]:
trace_names = list(max_index)
traces = []
for trace in trace_names:
    traces.append(go.Box(y=df_neighborhood_month[trace].values, name=trace))

data = traces
py.iplot(data)

In [ ]:
import json
import gmaps
import shapefile
import gmaps.geojson_geometries
gmaps.configure(api_key='XXXXXX')

In [ ]:
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
denver_coordinates = (39.7341699,-104.9257629)
#counties_geojson = gmaps.geojson_geometries.load_geometry('us-counties')

with open('crime.json') as f:
    geometry = json.load(f)

update_day = df_neighborhood.index.max()
time_span = datetime.timedelta(days=30)
thirty_days_ago = update_day - time_span
df_thirtyD = df[(df['DATE'] >= thirty_days_ago) & (df['IS_CRIME'] == 1)]
crime_location_thirtyD = df_thirtyD[['GEO_LAT', 'GEO_LON']].dropna()

crime_thirtyD_heatmap = gmaps.heatmap_layer(crime_location_thirtyD)

fig = gmaps.figure(map_type='ROADMAP', 
                   layout=figure_layout, 
                   center=denver_coordinates,
                   zoom_level=11)

#neighborhood_layer = gmaps.geojson_layer(geometry)
counties_layer = gmaps.geojson_layer(counties_geojson)
#fig.add_layer(counties_layer)
fig.add_layer(crime_thirtyD_heatmap)
crime_thirtyD_heatmap.point_radius = 11
crime_thirtyD_heatmap.max_intensity = 8
crime_thirtyD_heatmap.opacity = 0.35
fig

In [ ]:
from ipywidgets.embed import embed_minimal_html

embed_minimal_html('export.html', views=[fig])